In [1]:
# The model in "Implementation_2.ipynb" was found to lead to better results

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import matplotlib.pyplot as plt
import nltk

# Data Import and Processing

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd drive/MyDrive

/content/drive/MyDrive


In [5]:
! python -m spacy download en_core_web_md

2023-05-22 16:55:27.673054: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 20.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [6]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

In [7]:
newsgroups_train.data[0]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

In [8]:
nltk.download('stopwords')
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [10]:
def sent_to_words(sentences):
   for sentence in sentences:
      yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [11]:
import csv
import pandas as pd

In [12]:
tweets_csv = pd.read_csv('./final_tweets_set.csv')
tweets_csv

,Unnamed: 0,edit_history_tweet_ids,author_id,created_at,conversation_id,id,text,geo.place_id,referenced_tweets,city,inside,withheld.copyright,withheld.country_codes,withheld.scope,hashtag,text_hash_split,text_clean
0,1268,['1391443346443411457'],1267945445084598274,2021-05-09T17:22:00.000Z,1391443346443411457,1.391443e+18,Does #FortWorth ever cross your mind? \nhttps:...,NaN,NaN,Fort Worth,1.0,NaN,NaN,NaN,"['FortWorth', 'Stockyards', 'SUV', 'Limo']",Does fort worth ever cross your mind? \nhttps:...,does fort worth ever cross your mind and 469 2...
1,4672,['1311653244112957440'],5445832,2020-10-01T13:04:37.000Z,1311653244112957440,1.311653e+18,P.F.L. Always! 2004-06\n.\n.\nFollow @playarab...,42e46bc3663a4b5f,NaN,Fort Worth,1.0,NaN,NaN,NaN,"['playarabbit', 'theparksmall', 'fortworth', '...",P.F.L. Always! 2004-06\n.\n.\nFollow @playarab...,p f l always 06 follow play a rabbit the park ...
2,2327,['1423822631577456648'],41679815,2021-08-07T01:45:43.000Z,1423822631577456648,1.423823e+18,"Disabled vehicle, right lane blocked in #FortW...",42e46bc3663a4b5f,NaN,Fort Worth,1.0,NaN,NaN,NaN,"['FortWorth', 'DFWTraffic']","Disabled vehicle, right lane blocked in fort w...",disabled vehicle right lane blocked in fort wo...
3,66,['1287040376738635776'],347029713,2020-07-25T15:01:52.000Z,1287040376738635776,1.287040e+18,Pickup Man the Cowboy’s Cowboy🤠 #stockyardsch...,42e46bc3663a4b5f,NaN,Fort Worth,1.0,NaN,NaN,NaN,"['stockyardschampionshiprodeo', 'stockyardspro...",Pickup Man the Cowboy’s Cowboy🤠 stockyardscha...,pickup man the cowboys cowboy stockyardschampi...
4,2525,['1376576317903241217'],41679815,2021-03-29T16:45:44.000Z,1376576317903241217,1.376576e+18,Accident. two left lanes blocked. in #FortWort...,42e46bc3663a4b5f,NaN,Fort Worth,1.0,NaN,NaN,NaN,"['FortWorth', 'DFWTraffic']",Accident. two left lanes blocked. in fort wort...,accident two left lanes blocked in fort worth ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27195,107618,['1315221808325435393'],1099150592138596353,2020-10-11T09:24:49.000Z,1315221808325435393,1.315222e+18,Wearing the smile my momma gave me 💕💖👄⚡️🍓🧠💅🏼🍇 ...,1279db2db14240db,NaN,Los Angeles,0.0,NaN,NaN,NaN,"['SelfAddiction', 'LosAngeles']",Wearing the smile my momma gave me 💕💖👄⚡️🍓🧠💅🏼🍇 ...,wearing the smile my momma gave me self addict...
27196,108522,['1440294771709284363'],3526993403,2021-09-21T12:40:07.000Z,1440294771709284363,1.440295e+18,L.A. is Calling Again! 🎶 🎤 #singer #losangeles...,0051d76b1627f404,NaN,Los Angeles,0.0,NaN,NaN,NaN,"['singer', 'losangeles', 'newjersey', 'sony', ...",L.A. is Calling Again! 🎶 🎤 singer los angeles ...,l a is calling again singer los angeles newjer...
27197,107637,['1314770648904994820'],32148675,2020-10-10T03:32:04.000Z,1314770648904994820,1.314771e+18,So get ready #LosAngeles!\nUnless #Miami #Heat...,35f9d631a3650f85,NaN,Los Angeles,0.0,NaN,NaN,NaN,"['LosAngeles', 'Miami', 'Heat', 'LeBronJames',...",So get ready los angeles!\nUnless miami heat a...,so get ready los angeles unless miami heat att...
27198,107125,['1217178572462399489'],1196663730144075777,2020-01-14T20:16:00.000Z,1217178572462399489,1.217179e+18,That's the cop who threatened me worldstar @vi...,2da132a7bfebfc0c,NaN,Los Angeles,0.0,NaN,NaN,NaN,"['pig', 'montebello', 'montebellopd', 'LosAnge...",That's the cop who threatened me worldstar @vi...,that s the cop who threatened me worldstar vic...


In [13]:
### USE THE FOLLOWING IF THE DATA IS NOT ALREADY SAMPLED
# data = tweets_csv.loc[~(tweets_csv.text_clean.str.len() < 56)]
# seed = 12345
# samples = []

# for city in data.city.unique().tolist():
#     samples.append(data[(data.city == city) & (data.inside == 1)].sample(n=300, random_state=seed))
#     samples.append(data[(data.city == city) & (data.inside == 0)].sample(n=300, random_state=seed))

# data = pd.concat(samples).text_clean


# data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
# data = [re.sub('\s+', ' ', sent) for sent in data]
# data = [re.sub("\'", "", sent) for sent in data]

In [14]:
data_words = list(sent_to_words(tweets_csv.text_clean))

In [15]:
common_words = ['give', 'get', 'go', 'want', 'gets', 'getting', 'wants']

In [16]:
def remove_stopwords(texts):
   return [[word for word in simple_preprocess(str(doc)) 
   if word not in stop_words] for doc in texts]

def remove_common_words(texts):
   return [[word for word in simple_preprocess(str(doc)) 
   if word not in common_words] for doc in texts]

def make_bigrams(texts):
   return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
   return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
   texts_out = []
   for sent in texts:
      doc = nlp(" ".join(sent))
      texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
   return texts_out

In [17]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [18]:
data_words_nostops = remove_stopwords(data_words)
data_words_nocommon = remove_common_words(data_words_nostops)
data_words_bigrams = make_bigrams(data_words_nocommon)
data_lemmatized = lemmatization(
   data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
)

In [19]:
print(data_lemmatized[0])

['ever', 'cross', 'mind', 'ready', 'show', 'gateway', 'limo']


# Creating the Model

In [20]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]


In [21]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:4]]

[[('cross', 1),
  ('ever', 1),
  ('gateway', 1),
  ('limo', 1),
  ('mind', 1),
  ('ready', 1),
  ('show', 1)],
 [('always', 1), ('follow', 1), ('park', 1), ('play_rabbit', 1), ('small', 1)],
 [('block', 1), ('dfw_traffic', 1)],
 [('championship', 1), ('cowboy', 1), ('man', 2), ('pickup', 1), ('rodeo', 1)]]

In [22]:
lda_model = gensim.models.ldamodel.LdaModel(
   corpus=corpus, id2word=id2word, num_topics=15, random_state=100, 
   update_every=1, chunksize=100, passes=20, alpha='auto', per_word_topics=True
)

# Viewing Results

In [23]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.153*"come" + 0.108*"back" + 0.105*"let" + 0.059*"play" + 0.050*"old" + '
  '0.045*"visit" + 0.033*"birthday" + 0.029*"listen" + 0.027*"school" + '
  '0.024*"trip"'),
 (1,
  '0.361*"new" + 0.078*"art" + 0.070*"home" + 0.037*"beautiful" + '
  '0.032*"morning" + 0.031*"family" + 0.029*"ready" + 0.022*"house" + '
  '0.022*"close" + 0.021*"drop"'),
 (2,
  '0.665*"los_angele" + 0.032*"help" + 0.024*"know" + 0.016*"business_dentist" '
  '+ 0.016*"cbd_oil" + 0.016*"hair_salon" + 0.016*"restart" + '
  '0.016*"restaurants_veterinarian" + 0.016*"janitorial_service" + '
  '0.014*"philly"'),
 (3,
  '0.081*"love" + 0.059*"look" + 0.030*"fashion" + 0.029*"need" + '
  '0.029*"friend" + 0.026*"share" + 0.024*"available" + 0.024*"s" + '
  '0.023*"style" + 0.023*"call"'),
 (4,
  '0.067*"studio" + 0.064*"color" + 0.053*"ask" + 0.050*"tattoo" + '
  '0.039*"company" + 0.034*"sound" + 0.021*"ink" + 0.020*"pencil" + '
  '0.018*"pen" + 0.014*"skin"'),
 (5,
  '0.076*"music" + 0.064*"day" + 0.034*"see"

In [24]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))


Perplexity:  -15.49846179995724


In [25]:
coherence_model_lda = CoherenceModel(
   model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v'
)
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.33501288109744426


In [26]:
! pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 66.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.1 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.


In [27]:
! pip install "pandas<2.0.0"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 52.6 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.1
    Uninstalling pandas-2.0.1:
      Successfully uninstalled pandas-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyldavis 3.4.1 requires pandas>=2.0.0, but you have pandas 1.5.3 which is incompatible.


In [28]:
import pyLDAvis
import pyLDAvis.gensim

In [29]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.305923 -0.324440       1        1  24.040333
3     -0.154551  0.029215       2        1  13.130188
8     -0.092075  0.016564       3        1  10.380283
2     -0.261699  0.327319       4        1   9.311811
1     -0.061074  0.009711       5        1   7.735816
7      0.004941  0.000425       6        1   6.545961
12     0.021071 -0.001263       7        1   5.834793
14     0.035298 -0.002568       8        1   5.217623
0      0.057979 -0.004419       9        1   4.841128
6      0.081290 -0.006072      10        1   3.458167
11     0.099027 -0.007147      11        1   2.841163
9      0.137800 -0.009056      12        1   2.783169
10     0.132212 -0.008825      13        1   2.128418
13     0.156515 -0.009906      14        1   0.882969
4      0.149190 -0.009538      15        1   0.868178, topic_info=             Term          Freq         Total Category  logprob  loglift
2437   los_angele  15567.000000  15567.000000  Default  30.0000  30.0000
32            new   7015.000000   7015.000000  Default  29.0000  29.0000
662          good   1997.000000   1997.000000  Default  28.0000  28.0000
180         music   4612.000000   4612.000000  Default  27.0000  27.0000
167          come   1863.000000   1863.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
6796   watercolor      0.064431      1.496089  Topic15 -10.4301   1.6015
21028    inkdrawe      0.056152      1.486834  Topic15 -10.5676   1.4702
798          body      0.056511      1.506712  Topic15 -10.5612   1.4633
808           com      0.054314      1.501758  Topic15 -10.6009   1.4269
1722        email      0.053825      1.495243  Topic15 -10.6099   1.4222

[500 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
19       12  0.989408          accident
87       12  0.947966  accident_cleared
1535      3  0.998533             actor
7526     11  0.989517           actress
589       5  0.989277             album
...     ...       ...               ...
368       3  0.996999             woman
228       4  0.978116              word
121      10  0.997742              work
1673      1  0.998530             world
35        6  0.999305              year

[387 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 9, 3, 2, 8, 13, 15, 1, 7, 12, 10, 11, 14, 5])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# TF-IDF Analysis

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
import nltk
nltk.download()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> words


      Unzipping corpora/words.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [32]:
eng_words = set(nltk.corpus.words.words())

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
tweets_lst = []
for t in data_words:
    t_str = ' '.join(t)
    cleaned_t = " ".join(w for w in nltk.wordpunct_tokenize(t_str) if w.lower() in eng_words or not w.isalpha())
    tweets_lst.append(cleaned_t)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(tweets_lst)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import pandas as pd

train = tweets_lst

countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english')
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')

# convert th documents into a matrix
count_wm = countvectorizer.fit_transform(train)
tfidf_wm = tfidfvectorizer.fit_transform(train)
#retrieve the terms found in the corpora
# if we take same parameters on both Classes(CountVectorizer and TfidfVectorizer) , it will give same output of get_feature_names() methods)
#count_tokens = tfidfvectorizer.get_feature_names() # no difference
count_tokens = countvectorizer.get_feature_names_out()
tfidf_tokens = tfidfvectorizer.get_feature_names_out()
df_countvect = pd.DataFrame(data = count_wm.toarray(),columns = count_tokens)
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),columns = tfidf_tokens)
print("Count Vectorizer\n")
print(df_countvect)
print("\nTD-IDF Vectorizer\n")
print(df_tfidfvect)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Count Vectorizer

       a_luve_of_hair  aa  aba  abandoned  abate  abb  abbas  abbey  abbot  \
0                   0   0    0          0      0    0      0      0      0   
1                   0   0    0          0      0    0      0      0      0   
2                   0   0    0          0      0    0      0      0      0   
3                   0   0    0          0      0    0      0      0      0   
4                   0   0    0          0      0    0      0      0      0   
...               ...  ..  ...        ...    ...  ...    ...    ...    ...   
27195               0   0    0          0      0    0      0      0      0   
27196               0   0    0          0      0    0      0      0      0   
27197               0   0    0          0      0    0      0      0      0   
27198               0   0    0          0      0    0      0      0      0   
27199               0   0    0          0      0    0      0      0      0   

       abdication  ...  zinfandel  zip  zippe

In [49]:
def get_tfidf_index(word):
    return df_tfidfvect[df_tfidfvect['abbot']>0].index[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [50]:
word_of_interest = 'abbot'
get_tfidf_index(word_of_interest)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


25716